In [1]:
%tensorflow_version 2.8.0
import numpy as np
from tensorflow.keras import Model as Model_
from tensorflow.keras.layers import Input, ReLU, LSTM, Dense, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_model_remediation.min_diff.losses.mmd_loss as MMD
import tensorflow_model_remediation.min_diff.losses.adjusted_mmd_loss as adjustedMMD

# torch.manual_seed(0) # Set for testing purposes, please do not change!

print(tf.keras.__version__)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
2.8.0


In [2]:
!pip install tensorflow_model_remediation

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#Need only to be used with google colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
import re

class Dataset_Preprocessing:
    def __init__(self, dir_path, include_dimension = 2, sample_size = 50, total_classes = 17):
        
        #Dataset Directory path
        self.dir_path = dir_path
        
        #Which Dimension file to include, possible values: 2 and 3
        self.include_dimension = include_dimension
        
        #Total frames in one Sample
        self.sample_size = sample_size
        
        #Activity classes to include
        self.classes = ['SittingDown', 'Walking', 'Directions', 'Discussion', 'Sitting', 'Phoning', 'Eating', 'Posing', 'Greeting', 'Smoking']
        
        #Total activity classes
        self.total_classes = len(self.classes)
        
        #Subject Folders names in the Dataset
        self.internal_folders = ['S1', 'S5','S6','S7','S8','S9','S11']
    
    def read_dataset(self):
        try:
            #Contains all the different activity vectors
            activity_vector = {}
            
            #Contains the overall dataset
            sampled_data = None
            
            #Based on dimensions, which folder to use for extracting the dataset files
            data_folder = 'Poses_D2_Positions' if self.include_dimension == 2 else 'Poses_D3_Positions'
            
            #Checking if the dataset path is valid
            if not os.path.exists(self.dir_path):
                print('The Data Directory Does not Exist!')
                return None

            #Iterating over all the subject folders
            for fld in self.internal_folders:
                #Iterating for each file in the specified folder
                for file in os.listdir(os.path.join(self.dir_path, fld, data_folder)):
                    #Extracting the activity from the filename
                    activity = self.__extract_activity(file)
                    
                    if activity not in self.classes:
                        continue
                    
                    #Reading the CSV file using Pandas
                    data = pd.read_csv(os.path.join(self.dir_path, fld, data_folder, file), header=None)

                    #Formulating the activity vector using one hot encoding
                    if activity not in activity_vector:
                        total_keys = len(activity_vector.keys())
                        activity_vector[activity] = np.zeros(self.total_classes)
                        activity_vector[activity][total_keys] = 1
                    vector = activity_vector[activity]
                    
                    #Sampling the dataset
                    grouped_sample = self.__group_samples(data, self.sample_size, vector)
                    sampled_data = grouped_sample if sampled_data is None else np.append(sampled_data, grouped_sample, axis=0)
                    
            return sampled_data
        except Exception as e:
            print(e)
    
    def __extract_activity(self, filename):
        try:
            #Extracting the filename and excluding the extension
            name = os.path.splitext(filename)[0]
            
            #Substituting the empty string with characters other than english alphabets
            activity = re.sub('[^A-Za-z]+' , '' , name)
            return activity
        except Exception as e:
            print(e)
    
    def __group_samples(self, dataset, sample_size, activity):
        try:
            #Checking if the dataset is a Pandas Dataframe
            if not isinstance(dataset, pd.DataFrame):
                print('Expecting Pandas Dataframe, but got {}'.format(type(dataset)))
                return None
            
            #Appending activity class to each row in the dataset
            dataset = pd.concat([dataset, pd.DataFrame(np.tile(activity, (dataset.shape[0],1)))], axis=1)
            
            #Reshaping the dataset into sample batches
            total_samples = dataset.shape[0]//sample_size
            total_features = dataset.shape[1]
            grouped_rows = dataset.to_numpy()[:total_samples*self.sample_size].reshape((-1,self.sample_size, total_features))
            
            return grouped_rows
        except Exception as e:
            print(e)

In [5]:
#For long term prediction, we need a sample size of 60(10 frames input sequance, 50 frames predicted sequance)
sampled_data = Dataset_Preprocessing('/content/drive/MyDrive/Colab Notebooks/H3.6csv', sample_size=60).read_dataset()

In [6]:
#To make the data divisible for batch size of hunderd
total_batches = sampled_data.shape[0]
sampled_data = sampled_data[:total_batches-(total_batches%100)]

In [7]:
def split_to_features_labels(dataset, input_sequance_size=10) :
    """
    Function for splitting the data into features(with sequance size=iput_sequance_size)
    and labels which should be the remainder of the sample length 
    """
    assert input_sequance_size < dataset.shape[1], f"input sequance should be smaller than the total sample size"
    features = dataset[:, np.s_[0:input_sequance_size], :]
    labels = dataset[:,np.s_[input_sequance_size:], :64]
    
    return features, labels

In [8]:
sampled_dataX, sampled_dataY = split_to_features_labels(sampled_data, input_sequance_size=10)

In [9]:
print('Total Samples: {}'.format(sampled_dataY.shape[0]))
print('Total Frames: {}'.format(sampled_dataY.shape[1]))
print('Total Features: {}'.format(sampled_dataY.shape[2]))

Total Samples: 25500
Total Frames: 50
Total Features: 64


In [10]:
class InterpolationLayer(tf.keras.layers.Layer):
    """
    Custom interpolation layer extending the keras layer class
    it has one attribute num_frames to be interpolated between each two consecutive 
    timesteps
    it has one main function interpolateFrames  
    """
  
    def __init__(self, num_frames=5):
        super(InterpolationLayer, self).__init__()
        self.num_frames = num_frames
       
    def interpolateFrames(self, inputs):
      """
      Takes input tensors of shape(batch_size, timesteps, features)
      returns interpolated frames with shape(batch_size, timesteps*num_frames, features)
      """
      batch_size = inputs.shape[0]
      timesteps = inputs.shape[1]
      features = inputs.shape[2]
      #interpolated_frames = tf.zeros([batch_size, timesteps, 0, features])
      interpolated_frames = tf.zeros([0, features])

      for batch in range(batch_size) :
        for t in range(timesteps) :
          for j in range(self.num_frames) :
            X_i0 = inputs[batch, t]
            if(t == timesteps-1) :
              X_i1 = inputs[batch, t]
            else :  
              X_i1 = inputs[batch, t+1]
            alpha_j = j/self.num_frames
            current_frame = alpha_j*X_i0 + (1-alpha_j)*X_i1
            current_frame = tf.reshape(current_frame, [1, features])
            interpolated_frames = tf.concat((interpolated_frames, current_frame), axis=0)
            
      interpolated_frames = tf.reshape(interpolated_frames,
                                       [batch_size, (timesteps)*self.num_frames, features])
      return interpolated_frames

    def call(self, inputs):
      return self.interpolateFrames(inputs)

In [11]:
class GlocalNet(Model_):
    """
    A full GlocalNet implementation include the three main stages
    Glogen generating initial sparse frames
    Interpolation layer generating dense frames from Glogen output
    Locgen generating the final output by smoothing the interpolated frames
    """
    def __init__(self, enocder_hidden_state=200, decoder_hidden_state=200, 
                 output_diminsion=64, activation='relu', interpolation_frames=5):
        super(GlocalNet, self).__init__()
        #Glogen layers
        self.glogen_encoder = LSTM(enocder_hidden_state, return_state=True, return_sequences=True)
        self.glogen_decoder = LSTM(decoder_hidden_state, return_sequences=True, return_state=True)
        self.glogen_dense_layer = TimeDistributed(Dense(output_diminsion, activation=activation)) 
        
        #Interpolation layer
        self.interpolation_layer = InterpolationLayer(num_frames=interpolation_frames)

        #Locgen layers
        self.locgen_encoder = LSTM(enocder_hidden_state, return_sequences=True, return_state=True)
        self.locgen_decoder = LSTM(decoder_hidden_state, return_sequences=True, return_state=True)
        self.locgen_dense_layer = TimeDistributed(Dense(output_diminsion, activation=activation)) 
        
    def call(self, inputs):
        #Glogen calls      
        encoder_outputs, state_h, state_c = self.glogen_encoder(inputs)
        encoder_states = [state_h, state_c]
        output, _, _ = self.glogen_decoder(encoder_outputs, initial_state=encoder_states)
        glogen_output = self.glogen_dense_layer(output)

        #Interpolation call
        interpolated_frames = self.interpolation_layer(glogen_output)
        
        #Locgen calls
        locgen_encoder_outputs, locgen_state_h, locgen_state_c = self.locgen_encoder(interpolated_frames)
        locgen_encoder_states = [locgen_state_h, locgen_state_c]
        locgen_output, _, _ = self.locgen_decoder(locgen_encoder_outputs, initial_state=locgen_encoder_states)
        final_output = self.locgen_dense_layer(locgen_output)
        return final_output

In [12]:
class JointLoss() :
  """
  Joint loss class with two weight attributes for two different losses
  first one is the loss joint and the second is the loss_motion_flow
  """
  def __init__(self, lambda1=0.5, lambda2=0.5) :
    self.lambda1 = lambda1
    self.lambda2 = lambda2

  def loss_joint(self, predicted_sequance_batch, target_sequance_batch) :
      """
      Loss between the joint positions and its corresponding counterparts in the groundtruth
      """
      diff_norm_2 = tf.math.reduce_sum(tf.square(tf.subtract(predicted_sequance_batch, target_sequance_batch)), axis=2)
      return tf.reduce_sum(diff_norm_2, axis=1) 

  def loss_motion_flow(self, predicted_sequance_batch, target_sequance_batch) :
      """
      Loss between the motion flow of predicted sequance and the ground truth
      where the motion flow is the euclidean distance between each two consecutive frames
      """
      predictions_tomporal_diffs = tf.experimental.numpy.diff(predicted_sequance_batch, axis=1)
      real_tomporal_diffs = tf.experimental.numpy.diff(target_sequance_batch, axis=1)
      prediction_motion_flow_diff_norm_2 = tf.reduce_sum(tf.square(tf.subtract(predictions_tomporal_diffs, real_tomporal_diffs)), axis=2)
      return tf.reduce_sum(prediction_motion_flow_diff_norm_2, axis=1)


  def total_loss(self, target_sequance_batch, predicted_sequance_batch) :
      """
      calculating the total loss through a combination of the joint_loss and motion_flow_loss
      """
      joints_loss = self.loss_joint(predicted_sequance_batch, target_sequance_batch)
      motion_flow_loss = self.loss_motion_flow(predicted_sequance_batch, target_sequance_batch)
      return self.lambda1*joints_loss + self.lambda2*motion_flow_loss

In [13]:
def run_experiment(learning_rate=0.002, lambda1=0.5, lambda2=0.5, use_mse=False,
                   use_MMD=False, metrics=None, batch_size=100, epochs=50,
                   validation_split=0.2) :
  """
  Method takes all hyperparameters as input paramters and returns the model and history as
  a result
  """
  glocal_model = GlocalNet()
  if use_mse :
    loss_function = tf.keras.losses.mean_squared_error
  elif use_MMD :
    loss_function = adjustedMMD.AdjustedMMDLoss()
  else :
    loss_function = JointLoss(lambda1=lambda1, lambda2=lambda2).total_loss
  
  glocal_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                       loss=loss_function, metrics=metrics)
  history = glocal_model.fit(sampled_dataX, sampled_dataY,
          batch_size=batch_size,
          epochs=epochs)
  return history, glocal_model

## Experiment Running for MSE and joint loss

In [ ]:
history_mse, glocal_model_mse = run_experiment(epochs=10, use_mse=True, metrics=[tf.keras.losses.mean_absolute_percentage_error])

Epoch 1/10
255/255 [==============================] - 415s 240ms/step - loss: 197673.3438 - mean_absolute_percentage_error: 88.2866
Epoch 2/10
255/255 [==============================] - 63s 248ms/step - loss: 128336.4766 - mean_absolute_percentage_error: 67.4349
Epoch 3/10
255/255 [==============================] - 62s 243ms/step - loss: 82020.9609 - mean_absolute_percentage_error: 50.1100
Epoch 4/10
255/255 [==============================] - 64s 250ms/step - loss: 52302.0547 - mean_absolute_percentage_error: 37.0668
Epoch 5/10
255/255 [==============================] - 61s 240ms/step - loss: 34254.2656 - mean_absolute_percentage_error: 28.3228
Epoch 6/10
255/255 [==============================] - 61s 241ms/step - loss: 23976.3828 - mean_absolute_percentage_error: 23.0670
Epoch 7/10
255/255 [==============================] - 63s 245ms/step - loss: 18552.9414 - mean_absolute_percentage_error: 20.3186
Epoch 8/10
255/255 [==============================] - 61s 241ms/step - loss: 15939.6357

In [ ]:
history_jointLoss, glocal_model_jointLoss = run_experiment(epochs=10, lambda1=0.5, lambda2=0.5, metrics=[tf.keras.losses.mean_absolute_percentage_error])

Epoch 1/10
255/255 [==============================] - 419s 248ms/step - loss: 318235136.0000 - mean_absolute_percentage_error: 88.8408
Epoch 2/10
255/255 [==============================] - 63s 248ms/step - loss: 209204064.0000 - mean_absolute_percentage_error: 68.7124
Epoch 3/10
255/255 [==============================] - 64s 252ms/step - loss: 136290848.0000 - mean_absolute_percentage_error: 52.0030
Epoch 4/10
255/255 [==============================] - 63s 248ms/step - loss: 89421984.0000 - mean_absolute_percentage_error: 39.3298
Epoch 5/10
255/255 [==============================] - 64s 251ms/step - loss: 60899012.0000 - mean_absolute_percentage_error: 30.7696
Epoch 6/10
255/255 [==============================] - 65s 254ms/step - loss: 44627976.0000 - mean_absolute_percentage_error: 25.5873
Epoch 7/10
255/255 [==============================] - 64s 250ms/step - loss: 36028060.0000 - mean_absolute_percentage_error: 22.8682
Epoch 8/10
255/255 [==============================] - 64s 251ms/s

## Running experiment with MMD

In [18]:
history_mmd, glocal_model_mmd = run_experiment(epochs=10, use_MMD=True, metrics=[tf.keras.losses.mean_absolute_percentage_error])

Epoch 1/10


ValueError: ignored

In [14]:
glocalNet_model = GlocalNet()

In [15]:
predictions = glocalNet_model(sampled_dataX[:100])

In [18]:
print(predictions.shape)
print(sampled_dataY[:100].shape)

(100, 50, 64)
(100, 50, 64)


In [19]:
mmd_loss = MMD.MMDLoss()

In [ ]:
mmd_loss(tf.reshape(predictions, [100, 50*64]),tf.reshape(sampled_dataY[:100], [100, 50*64]))